# **📘 Aplikasi Scraping Buku Otomatis dari Google Books API**


*Zahwa Genoveva - 51421554*

*Class: 4IA12*



<center>
    <img src="https://i.pinimg.com/736x/ee/fb/8b/eefb8b0f338617d38a664e4319703c16.jpg" width="10500" height="290"/>
</center>

url : [link text book API](https://books.google.co.id/)

In [ ]:
import requests
import csv
import json
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Variabel global untuk menyimpan data buku hasil scraping
hasil_buku_global = []

# Ambil data dari Google Books API
def get_books_data(keyword):
    books = []
    try:
        for start in range(0, 100, 40):  # Ambil dalam batch
            params = {
                'q': keyword,
                'startIndex': start,
                'maxResults': 40,
                'printType': 'books',
                'langRestrict': 'id'
            }
            response = requests.get('https://www.googleapis.com/books/v1/volumes', params=params)
            data = response.json()
            items = data.get('items', [])
            for item in items:
                volume = item.get('volumeInfo', {})
                image_links = volume.get('imageLinks', {})
                book = {
                    'judul': volume.get('title', 'Tidak tersedia'),
                    'penulis': ', '.join(volume.get('authors', ['Tidak diketahui'])),
                    'tanggal': volume.get('publishedDate', 'Tidak tersedia'),
                    'penerbit': volume.get('publisher', 'Tidak tersedia'),
                    'deskripsi': volume.get('description', 'Tidak tersedia')[:300] + '...',
                    'sampul': image_links.get('thumbnail', 'https://via.placeholder.com/128x180?text=No+Image')
                }
                books.append(book)
            if len(books) >= 100:
                break
    except Exception as e:
        books = [{'judul': f"Terjadi kesalahan: {e}"}]
    return books[:100]

In [ ]:
# Format HTML per buku
def tampilkan_buku_rapi(buku_list):
    html_output = ""
    for i, b in enumerate(buku_list, 1):
        html_output += f"""
        <div style="display: flex; border: 1px solid #ccc; border-radius: 10px; padding: 15px; margin: 10px 0; background-color: #fefefe; box-shadow: 2px 2px 8px rgba(0,0,0,0.05);">
            <img src="{b['sampul']}" alt="Sampul" style="width:128px; height:auto; border-radius:5px; margin-right:15px;" />
            <div style="flex: 1;">
                <h3 style="margin-bottom: 5px; color: #2b4c7e;">{i}. {b['judul']}</h3>
                <pre style="font-size: 14px; line-height: 1.6; color: #333; background-color: #f9f9f9; padding: 10px; border-radius: 5px;">
   Penulis        : {b['penulis']}
   Tanggal Terbit : {b['tanggal']}
   Penerbit       : {b['penerbit']}
   Deskripsi      : {b['deskripsi']}
                </pre>
            </div>
        </div>
        """
    return html_output

# Fungsi untuk simpan ke CSV
def simpan_csv(b):
    if hasil_buku_global:
        with open('buku_output.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=hasil_buku_global[0].keys())
            writer.writeheader()
            writer.writerows(hasil_buku_global)
        print("✅ Data berhasil disimpan sebagai 'buku_output.csv'.")

In [ ]:
# Fungsi untuk simpan ke JSON
def simpan_json(b):
    if hasil_buku_global:
        with open('buku_output.json', 'w', encoding='utf-8') as f:
            json.dump(hasil_buku_global, f, indent=4, ensure_ascii=False)
        print("✅ Data berhasil disimpan sebagai 'buku_output.json'.")


In [ ]:
# Komponen GUI
keyword_input = widgets.Text(
    value='dongeng',
    placeholder='Masukkan kata kunci pencarian buku...',
    description='Kata Kunci:',
    layout=widgets.Layout(width='100%')
)

output_area = widgets.Output()
button_area = widgets.HBox()  # Akan menampilkan tombol simpan setelah pencarian

In [ ]:
# Fungsi pencarian
def saat_diklik(b):
    global hasil_buku_global
    with output_area:
        clear_output()
        print(f"Mengambil 100 data buku dengan kata kunci: '{keyword_input.value}'\n")
        buku = get_books_data(keyword_input.value)
        hasil_buku_global = buku  # simpan untuk keperluan export
        html = tampilkan_buku_rapi(buku)
        display(HTML(html))
        tampilkan_tombol_simpan()

# Fungsi menampilkan tombol simpan
def tampilkan_tombol_simpan():
    tombol_csv = widgets.Button(description="💾 Simpan ke CSV", button_style='info')
    tombol_json = widgets.Button(description="💾 Simpan ke JSON", button_style='primary')
    tombol_csv.on_click(simpan_csv)
    tombol_json.on_click(simpan_json)
    button_area.children = [tombol_csv, tombol_json]

# Tombol cari
tombol_cari = widgets.Button(description="Cari Buku 📚", button_style='success', layout=widgets.Layout(width='30%'))
tombol_cari.on_click(saat_diklik)


In [ ]:
# Desain ulang GUI akhir
title = widgets.HTML("""
<h2 style='
    color: white;
    background-color: #4a4a4a;
    padding: 15px;
    border-radius: 10px;
    text-align: center;
'>
📖 Aplikasi Scraping Buku dari Google Books API
</h2>
""")

subtext = widgets.HTML("""
<p style='
    font-size: 14px;
    color: white;
    background-color: #6e6e6e;
    padding: 10px;
    border-radius: 10px;
    text-align: center;
'>
Masukkan kata kunci untuk menampilkan 100 buku berdasarkan topik yang kamu cari.
</p>
""")

# Membungkus elemen GUI
container = widgets.VBox(
    [title, subtext, keyword_input, tombol_cari, button_area, output_area],
    layout=widgets.Layout(
        border='1px solid #ccc',
        padding='20px',
        border_radius='10px',
        background_color='#2e2e2e',
        box_shadow='0px 4px 10px rgba(0, 0, 0, 0.2)'
    )
)

display(container)